In [0]:
%python
%pip install sgp4

In [0]:
# Requires: pip install sgp4 numpy scipy pandas
import numpy as np
import scipy.linalg as la
from sgp4.api import Satrec, jday, WGS72
from datetime import datetime
import math

# ----------------- helpers -----------------
def get_position_tle(line1, line2, when_iso):
    """
    Return ECI position (km) using sgp4 at ISO time (UTC string 'YYYY-MM-DDTHH:MM:SS').
    """
    sat = Satrec.twoline2rv(line1, line2, WGS72)
    ts = datetime.fromisoformat(when_iso)
    jd, fr = jday(ts.year, ts.month, ts.day, ts.hour, ts.minute, ts.second + ts.microsecond/1e6)
    e, r, v = sat.sgp4(jd, fr)
    if e != 0:
        raise RuntimeError(f"SGP4 error {e} for {when_iso}")
    return np.array(r), np.array(v)  # r(km), v(km/s)

def build_combined_covariance_diag(sig_a_km3, sig_b_km3):
    """Make simple diagonal combined covariance from 3-vector sigmas (km)."""
    s1 = np.asarray(sig_a_km3).reshape(3)
    s2 = np.asarray(sig_b_km3).reshape(3)
    return np.diag(s1**2 + s2**2)

def estimate_pc_mc(rel_pos_km, cov_rel_km2, hbr_km, n_samples=200_000, seed=0):
    rng = np.random.default_rng(seed)
    jitter = 1e-12 * np.trace(cov_rel_km2) if np.isfinite(np.trace(cov_rel_km2)) else 1e-12
    cov = cov_rel_km2 + np.eye(3) * jitter
    try:
        L = la.cholesky(cov, lower=True)
    except la.LinAlgError:
        evals, evecs = la.eigh(cov)
        evals_clipped = np.clip(evals, 0, None)
        L = (evecs * np.sqrt(evals_clipped)) @ evecs.T
    z = rng.standard_normal(size=(n_samples, 3))
    samples = rel_pos_km.reshape(1,3) + z @ L.T
    d = np.linalg.norm(samples, axis=1)
    hits = (d <= hbr_km)
    p = hits.mean()
    stderr = math.sqrt(p*(1-p)/n_samples) if n_samples>0 else 0.0
    return float(p), float(stderr)

# ----------------- main routine -----------------
def compute_pc_for_pair(
    line1_a, line2_a, line1_b, line2_b,
    tca_iso,
    # optional: covariances or sigmas; prefer COV matrices in km^2
    covA_km2=None, covB_km2=None,
    # if covariances not provided, provide 3-element sigma vectors (radial, along, cross) in km
    sigmaA_km=None, sigmaB_km=None,
    # radii
    radiusA_m=1.0, radiusB_m=1.0,
    n_samples=200000
):
    # propagate to TCA
    rA, vA = get_position_tle(line1_a, line2_a, tca_iso)
    rB, vB = get_position_tle(line1_b, line2_b, tca_iso)
    rel = rA - rB  # mean relative vector (km)

    # build combined covariance
    if covA_km2 is not None and covB_km2 is not None:
        C = np.asarray(covA_km2, dtype=float) + np.asarray(covB_km2, dtype=float)
    else:
        # fall back to diagonal from sigma vectors
        if sigmaA_km is None: sigmaA_km = np.array([0.05, 0.5, 0.05])  # default LEO: rad, along, cross (km)
        if sigmaB_km is None: sigmaB_km = np.array([0.05, 0.5, 0.05])
        C = build_combined_covariance_diag(sigmaA_km, sigmaB_km)

    hbr_km = (radiusA_m + radiusB_m) / 1000.0

    # Monte Carlo estimate
    p, se = estimate_pc_mc(rel, C, hbr_km, n_samples=n_samples)
    return {
        "rel_km": rel,
        "relative_speed_km_s": np.linalg.norm(vA - vB),
        "miss_distance_km": np.linalg.norm(rel),
        "Pc": p,
        "Pc_std_err": se,
        "hbr_km": hbr_km,
        "C_trace_km2": float(np.trace(C))
    }


In [0]:
# fetch TLE lines for NORADs from your Delta table or TLE source
line1_21419 = "1 21419U 91041B   25232.45386839  .00000070  00000-0  33255-4 0  9991"   # replace with real TLE line1
line2_21419 = "2 21419  74.0370 234.7926 0039642 154.6600 205.6508 14.35100018788247"   # replace
line1_58074 = "1 58074U 23159B   25232.60847776  .00000131  00000-0  64456-4 0  9997"   # replace
line2_58074 = "2 58074  98.4722 248.9531 0100408  94.9791 266.2862 14.29140183 96473"   # replace

tca = "2025-08-23T12:34:56"  # use CDM TCA or your computed TCA

res = compute_pc_for_pair(
    line1_21419, line2_21419, line1_58074, line2_58074,
    tca_iso=tca,
    # if you have CDM covariance matrices, pass them here as 3x3 arrays (in km^2)
    covA_km2=None, covB_km2=None,
    # otherwise you can pass realistic sigma vectors (radial, along-track, cross-track) in km:
    sigmaA_km=np.array([0.02, 0.2, 0.02]),
    sigmaB_km=np.array([0.02, 0.2, 0.02]),
    radiusA_m=1.0, radiusB_m=1.0,
    n_samples=200000
)
print(res)
